# BiomeFUN 
# Stoichiometric Modelling

This notebook contains the exercises covered in the lecture on Stoichiometric Metabolic Modelling by Timmy Paez Watson. Some lines of code have been removed to be completed during the lecture. For access to the complete notebooks, please go to: https://tp-watson.github.io/Teaching-Site/materials/
## Model simulations with COBRA: E coli core model

In [1]:
import cobra
from cobra.io import load_json_model

# Load the Core E coli model - saved in the same folder
model = load_json_model("Ecoli_core.json")


# Custom functions to check solutions of FBA

def metabolite_flux_summary(model, met_id, solution):
    """
    For a given metabolite and FBA solution, print net production/consumption
    by each reaction.

    Parameters:
    - model: COBRA model
    - met_id: metabolite ID (e.g., 'atp_c')
    - solution: a solution object from model.optimize()
    """
    try:
        met = model.metabolites.get_by_id(met_id)
    except KeyError:
        print(f"❌ Metabolite '{met_id}' not found in model.")
        return

    print(f"\n🔬 Net flux contribution for '{met_id}' ({met.name}):")

    producers = []
    consumers = []

    for rxn in met.reactions:
        coeff = rxn.metabolites[met]         # stoichiometry
        flux = solution.fluxes[rxn.id]       # actual flux value
        net = coeff * flux                   # net production/consumption

        if net > 0:
            producers.append((rxn.id, net))
        elif net < 0:
            consumers.append((rxn.id, net))

    # Sort by absolute value of net flux
    producers.sort(key=lambda x: abs(x[1]), reverse=True)
    consumers.sort(key=lambda x: abs(x[1]), reverse=True)

    print("\n✅ Produced by:")
    for rxn_id, net in producers:
        print(f"  {rxn_id:25s} → +{net:.3f} mmol/gDW/h")

    print("\n❌ Consumed by:")
    for rxn_id, net in consumers:
        print(f"  {rxn_id:25s} → {net:.3f} mmol/gDW/h")


## Explore the model basics

In [2]:
# Explore model
print(f"Reactions: {len(model.reactions)}")
print(f"Metabolites: {len(model.metabolites)}")
print(f"Genes: {len(model.genes)}")

# Show some reactions
for r in list(model.reactions)[:5]:
    print(r.id, ":", r.reaction)

Reactions: 95
Metabolites: 72
Genes: 137
PFK : atp_c + f6p_c --> adp_c + fdp_c + h_c
PFL : coa_c + pyr_c --> accoa_c + for_c
PGI : g6p_c <=> f6p_c
PGK : 3pg_c + atp_c <=> 13dpg_c + adp_c
PGL : 6pgl_c + h2o_c --> 6pgc_c + h_c


## Let's start by maximizing growth with oxygen

In [3]:
print(model.objective)

Maximize
1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5


In [5]:
# Set glucose uptake
model.reactions.get_by_id("EX_glc__D_e").lower_bound = - 10       # TO BE COMPLETED 

# Solve FBA
sol_aerobic = model.optimize()
print(sol_aerobic.status)
print("Aerobic growth:", sol_aerobic.objective_value)



optimal
Aerobic growth: 0.8739215069684311


In [9]:
# Use our custom function to check what reactions use/produce O2 and ATP
metabolite_flux_summary(model, "o2_e", sol_aerobic)
metabolite_flux_summary(model, "atp_c", sol_aerobic)


🔬 Net flux contribution for 'o2_e' (O2 O2):

✅ Produced by:
  EX_o2_e                   → +21.799 mmol/gDW/h

❌ Consumed by:
  O2t                       → -21.799 mmol/gDW/h

🔬 Net flux contribution for 'atp_c' (ATP C10H12N5O13P3):

✅ Produced by:
  ATPS4r                    → +45.514 mmol/gDW/h
  PGK                       → +16.024 mmol/gDW/h
  SUCOAS                    → +5.064 mmol/gDW/h
  PYK                       → +1.758 mmol/gDW/h

❌ Consumed by:
  BIOMASS_Ecoli_core_w_GAM  → -52.269 mmol/gDW/h
  ATPM                      → -8.390 mmol/gDW/h
  PFK                       → -7.477 mmol/gDW/h
  GLNS                      → -0.223 mmol/gDW/h


## Now let's try to simulate anaerobic growth

In [12]:
# Limit oxygen *uptake* from environment
model.reactions.get_by_id("EX_o2_e").lower_bound = 0  # TO BE COMPLETED

# Solve FBA for growth 
sol_ferm = model.optimize()
print(sol_ferm.status)
print("Fermentative Growth:", sol_ferm.objective_value)


metabolite_flux_summary(model, "o2_e", sol_ferm)
metabolite_flux_summary(model, "atp_c", sol_ferm)

optimal
Fermentative Growth: 0.21166294973531055

🔬 Net flux contribution for 'o2_e' (O2 O2):

✅ Produced by:

❌ Consumed by:

🔬 Net flux contribution for 'atp_c' (ATP C10H12N5O13P3):

✅ Produced by:
  PGK                       → +19.437 mmol/gDW/h
  ACKr                      → +8.504 mmol/gDW/h
  PYK                       → +8.404 mmol/gDW/h

❌ Consumed by:
  BIOMASS_Ecoli_core_w_GAM  → -12.660 mmol/gDW/h
  PFK                       → -9.789 mmol/gDW/h
  ATPM                      → -8.390 mmol/gDW/h
  ATPS4r                    → -5.452 mmol/gDW/h
  GLNS                      → -0.054 mmol/gDW/h


## Explore underlying assumptions

Open questions you can explore to further understand the model

- Where is the energy being used up in?
- Is one of the reactions special / strange?
- What are the main fermentation products from the anaerobic growth?
- How strongly is growth affected by maintenance? - reaction ID "ATPM"